In [23]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.stem import SnowballStemmer
from collections import Counter
import regex  
import os

In [2]:
#ensure tokenizer is available
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cmetzing\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
#increase print limits 
pd.set_option('display.max_rows', None)

In [4]:
#read data
df = pd.read_csv(r'C:/Users/cmetzing/Downloads/b0cd514b-b9cc-4972-a0c2-c91726e6d825.csv')

In [5]:
#check out data
df.head(1)

,question_id,question_user_id,question_language,question_content,question_topic,question_sent,response_id,response_user_id,response_language,response_content,...,question_user_country_code,question_user_gender,question_user_dob,question_user_created_at,response_user_type,response_user_status,response_user_country_code,response_user_gender,response_user_dob,response_user_created_at
0,3849056,519124,nyn,E ABA WEFARM OFFICES ZABO NIZISHANGWA NKAHI?,NaN,2017-11-22 12:25:03+00,20691011,200868,nyn,E!23 Omubazi Ni Dudu Cipa',...,ug,NaN,NaN,2017-11-18 13:09:11+00,farmer,live,ug,NaN,NaN,2017-05-09 09:19:33+00


In [6]:
df.dtypes

question_id                    int64
question_user_id               int64
question_language             object
question_content              object
question_topic                object
question_sent                 object
response_id                    int64
response_user_id               int64
response_language             object
response_content              object
response_topic                object
response_sent                 object
question_user_type            object
question_user_status          object
question_user_country_code    object
question_user_gender          object
question_user_dob             object
question_user_created_at      object
response_user_type            object
response_user_status          object
response_user_country_code    object
response_user_gender          object
response_user_dob             object
response_user_created_at      object
dtype: object

In [7]:
#possible language values 
df['question_language'].value_counts()

question_language
eng    11976781
swa     6480848
nyn     1157228
lug      689986
Name: count, dtype: int64

In [8]:
#extract eng data
dfe = df[df['question_language'] == 'eng']

In [9]:
#confirm extraction 
dfe['question_language'].value_counts()

question_language
eng    11976781
Name: count, dtype: int64

In [10]:
dfe.head(5)

,question_id,question_user_id,question_language,question_content,question_topic,question_sent,response_id,response_user_id,response_language,response_content,...,question_user_country_code,question_user_gender,question_user_dob,question_user_created_at,response_user_type,response_user_status,response_user_country_code,response_user_gender,response_user_dob,response_user_created_at
1,3849061,521327,eng,Q this goes to wefarm. is it possible to get f...,NaN,2017-11-22 12:25:05+00,4334249,526113,eng,Q1 which stage is marleks last vaccinated,...,ug,NaN,NaN,2017-11-20 11:55:48+00,farmer,zombie,ug,NaN,NaN,2017-11-22 10:13:03+00
9,3849084,6642,eng,Q-i have stock rabbit's urine for 5 weeks mash...,rabbit,2017-11-22 12:25:10+00,3852272,35690,eng,Q165#Ksh120,...,ke,NaN,NaN,2015-07-28 17:12:04+00,farmer,zombie,ke,NaN,NaN,2015-11-14 19:59:19+00
15,3849098,526375,eng,Q J Have Mi 10000 Can J Start Aproject Of Pout...,poultry,2017-11-22 12:25:12+00,3859675,522795,eng,Q5 10000 is not enough because the materials n...,...,ug,NaN,NaN,2017-11-22 12:10:37+00,farmer,zombie,ug,NaN,NaN,2017-11-21 07:08:05+00
16,3849100,237506,eng,WHERE DO I GET SEEDS OF COCONUT?,pig,2017-11-22 12:25:12+00,4263505,412335,eng,"Q11:try kisumu ,migori ,homa bay,siaya and any...",...,ke,NaN,NaN,2017-06-27 10:45:20+00,farmer,destroyed,ke,NaN,NaN,2017-10-03 17:35:12+00
17,3849100,237506,eng,WHERE DO I GET SEEDS OF COCONUT?,coconut,2017-11-22 12:25:12+00,3852604,412335,eng,Q11:coast Cereals,...,ke,NaN,NaN,2017-06-27 10:45:20+00,farmer,destroyed,ke,NaN,NaN,2017-10-03 17:35:12+00


In [11]:
#reduce dataset to extract question since single question may have multiple responses 
dfe = dfe[["question_id", "question_user_id","question_content","question_topic","question_sent","question_user_country_code"]]

In [12]:
#keep only one row per unique question
dfe = dfe.drop_duplicates(subset="question_id", keep="first")

In [13]:
#inspect
dfe.head()

,question_id,question_user_id,question_content,question_topic,question_sent,question_user_country_code
1,3849061,521327,Q this goes to wefarm. is it possible to get f...,NaN,2017-11-22 12:25:05+00,ug
9,3849084,6642,Q-i have stock rabbit's urine for 5 weeks mash...,rabbit,2017-11-22 12:25:10+00,ke
15,3849098,526375,Q J Have Mi 10000 Can J Start Aproject Of Pout...,poultry,2017-11-22 12:25:12+00,ug
16,3849100,237506,WHERE DO I GET SEEDS OF COCONUT?,pig,2017-11-22 12:25:12+00,ke
23,3849129,54426,Q#.Which plant has omega3?,plant,2017-11-22 12:25:16+00,ke


In [14]:
dfe.describe(include="all")

,question_id,question_user_id,question_content,question_topic,question_sent,question_user_country_code
count,2.940542e+06,2.940542e+06,2940542,2218544,2940542,2940542
unique,NaN,NaN,2708132,148,2939086,4
top,NaN,NaN,What is crop rotation?,chicken,2018-02-01 17:55:53+00,ke
freq,NaN,NaN,839,265257,7,1665002
mean,2.931697e+07,1.659636e+06,NaN,NaN,NaN,NaN
std,1.615944e+07,1.009599e+06,NaN,NaN,NaN,NaN
min,3.849061e+06,7.000000e+00,NaN,NaN,NaN,NaN
25%,1.533666e+07,8.940460e+05,NaN,NaN,NaN,NaN
50%,2.762343e+07,1.428648e+06,NaN,NaN,NaN,NaN
75%,4.305158e+07,2.477375e+06,NaN,NaN,NaN,NaN


In [15]:
#check countries for where question user
dfe['question_user_country_code'].value_counts()

question_user_country_code
ke    1665002
ug    1275419
gb        118
tz          3
Name: count, dtype: int64

In [16]:
#check for nulls
dfe.isnull().sum()  #looks like we are missing question topic a lot

question_id                        0
question_user_id                   0
question_content                   0
question_topic                721998
question_sent                      0
question_user_country_code         0
dtype: int64

In [17]:
#missing
(721998/2940542)*100	

24.55322862247844

In [18]:
#check question topics
dfe['question_topic'].value_counts()

question_topic
chicken               265257
cattle                253285
maize                 200532
tomato                136091
plant                 123615
pig                    88258
bean                   81836
crop                   78338
banana                 70591
poultry                65525
potato                 56107
rabbit                 54606
cabbage                52850
animal                 49112
goat                   48781
coffee                 45644
onion                  33328
watermelon             26331
sheep                  23848
kale                   23210
wheat                  21402
bee                    21112
livestock              21106
passion-fruit          19541
cassava                18538
peanut                 17918
tree                   17811
tea                    17310
rice                   17176
fish                   15218
sugar-cane             14485
carrot                 13845
bird                   12240
grass                  11950

In [19]:
#basic word stem counting setup
stemmer = SnowballStemmer("english")
counter = Counter()

def process_text(text):
    #lowercase and remove non-alphabetic characters
    tokens = regex.findall(r'\b\w+\b', str(text).lower())
    stems = [stemmer.stem(token) for token in tokens]
    return stems

In [20]:
# Process each row and update stem counts
for row in dfe['question_content']:
    stems = process_text(row)
    counter.update(stems)

In [21]:
counter

Counter({'q': 1549075,
         'is': 964418,
         'i': 962786,
         'what': 938189,
         'to': 896895,
         'the': 884224,
         'of': 880628,
         'how': 699114,
         'can': 685831,
         'in': 672002,
         'which': 473894,
         'a': 462172,
         'my': 364652,
         'do': 362454,
         'and': 354005,
         'for': 353014,
         'plant': 339330,
         'are': 272405,
         'it': 252534,
         'best': 244175,
         'maiz': 240688,
         'use': 226725,
         'be': 186732,
         'cow': 186037,
         'have': 166749,
         'type': 157318,
         'get': 155301,
         'that': 153705,
         'tomato': 152130,
         'on': 147548,
         'farm': 141594,
         'ask': 139700,
         'where': 131389,
         'when': 116454,
         'doe': 115863,
         'one': 114972,
         'mani': 114723,
         'by': 114275,
         'from': 112289,
         'grow': 107712,
         'me': 107331,
         'go

In [24]:
#write counts to csv
counts = pd.DataFrame(counter.items(), columns=['item', 'count'])
counts.to_csv(os.path.join(os.path.dirname(os.getcwd()), 'data', 'allstemcounter_output.csv'), index=False)

In [25]:
#manual inspect specific key stem
target_stem = "unprofit"
matching_questions = []

for row in dfe['question_content']:
    stems = process_text(row)
    if target_stem in stems:
        matching_questions.append(row)

# Now matching_questions contains all question_contents with the chosen stem
for q in matching_questions:
    print(q)

Q In usin-Gishu maize farming as become unprofitable which other crops can u advise the farmers of uasin-gishu to grow
q which crop is unprofitable in uganda?
s if kept over that age by many weeks they are likely to be unprofitable.the enterprise must be organised to have the contracted number ready for marketi
Q what makes layers poultry unprofitable to keep
Q what makes layers poultry unprofitable to keep
Q A farmer asks: what makes layers poultry unprofitable to keep?
Jina asks: A farmer asks: what makes layers poultry unprofitable to keep?
Q88:Jina asks: A farmer asks: what makes layers poultry unprofitable to keep?


In [26]:
#these last 5 questions seems to be others asking the same question on behalf of the original asker not for their own information so we need to address this

In [28]:
#load top financial stems selected
stemmed_keywords = pd.read_csv(os.path.join(os.path.dirname(os.getcwd()), 'data', 'toppotentialfinancialkeywords.csv'),usecols=[0], skiprows=1, header=None)[0]
stemmed_keywords_set = set(stemmed_keywords.astype(str))

In [29]:
stemmed_keywords

0           price
1          market
2            cost
3            sell
4             buy
5           yield
6           money
7           order
8            sale
9          profit
10          capit
11           loan
12           cash
13          buyer
14           rate
15       interest
16           loss
17         invest
18            pay
19        amarket
20           fund
21          aloan
22          webuy
23           bank
24         borrow
25         buyend
26         credit
27        payment
28       contract
29         budget
30         seller
31           lend
32       cashcrop
33       moneymak
34         acapit
35          grant
36           abuy
37           ibuy
38           2buy
39         refund
40            tax
41         yieldz
42    supermarket
43        aprofit
44          repay
45         sellng
46          acash
47       investor
48        yielder
49           debt
50          4sale
51          acost
52         capita
53      highyield
54         prepay
55        

In [30]:
#check for top financial stems

def contains_stemmed_keyword(text):
    tokens = regex.findall(r'\b\w+\b', str(text).lower())
    stems = set(stemmer.stem(token) for token in tokens)
    return bool(stems & stemmed_keywords_set)  

dfe['contains_financial_word'] = dfe['question_content'].apply(contains_stemmed_keyword)

In [31]:
dfe.head()

,question_id,question_user_id,question_content,question_topic,question_sent,question_user_country_code,contains_financial_word
1,3849061,521327,Q this goes to wefarm. is it possible to get f...,NaN,2017-11-22 12:25:05+00,ug,True
9,3849084,6642,Q-i have stock rabbit's urine for 5 weeks mash...,rabbit,2017-11-22 12:25:10+00,ke,False
15,3849098,526375,Q J Have Mi 10000 Can J Start Aproject Of Pout...,poultry,2017-11-22 12:25:12+00,ug,False
16,3849100,237506,WHERE DO I GET SEEDS OF COCONUT?,pig,2017-11-22 12:25:12+00,ke,False
23,3849129,54426,Q#.Which plant has omega3?,plant,2017-11-22 12:25:16+00,ke,False


In [34]:
#Add a month column
dfe['question_sent'] = pd.to_datetime(dfe['question_sent'], errors='coerce')
dfe['month'] = dfe['question_sent'].dt.to_period('M').astype(str)

C:\Users\cmetzing\AppData\Local\Temp\ipykernel_136460\1418378379.py:3: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  dfe['month'] = dfe['question_sent'].dt.to_period('M').astype(str)


In [50]:
dfe.head()

,question_id,question_user_id,question_content,question_topic,question_sent,question_user_country_code,contains_financial_word,month
1,3849061,521327,Q this goes to wefarm. is it possible to get f...,NaN,2017-11-22 12:25:05+00:00,ug,True,2017-11
9,3849084,6642,Q-i have stock rabbit's urine for 5 weeks mash...,rabbit,2017-11-22 12:25:10+00:00,ke,False,2017-11
15,3849098,526375,Q J Have Mi 10000 Can J Start Aproject Of Pout...,poultry,2017-11-22 12:25:12+00:00,ug,False,2017-11
16,3849100,237506,WHERE DO I GET SEEDS OF COCONUT?,pig,2017-11-22 12:25:12+00:00,ke,False,2017-11
23,3849129,54426,Q#.Which plant has omega3?,plant,2017-11-22 12:25:16+00:00,ke,False,2017-11


In [51]:
dfef =  dfe[dfe['contains_financial_word'] == True].copy()

In [52]:
dfef.head()

,question_id,question_user_id,question_content,question_topic,question_sent,question_user_country_code,contains_financial_word,month
1,3849061,521327,Q this goes to wefarm. is it possible to get f...,NaN,2017-11-22 12:25:05+00:00,ug,True,2017-11
26,3849153,340091,Q Am Jackson From Ibanda If Want To Grow Tomat...,tomato,2017-11-22 12:25:18+00:00,ug,True,2017-11
47,3849285,262078,Q.How much is price of 1kg of onions farmgate ...,onion,2017-11-22 12:34:40+00:00,ke,True,2017-11
156,3849758,522802,Q why is the price of maize low this days,maize,2017-11-22 13:15:35+00:00,ug,True,2017-11
159,3849788,522802,Qwhat is the exact price of a kilo gram of ma...,maize,2017-11-22 13:19:13+00:00,ug,True,2017-11


In [53]:
# Ensure pandas copy on write is enabled
pd.options.mode.copy_on_write = True

# Calculate totals
total_questions = len(dfe)
financial_questions = len(dfef)

In [57]:
print(financial_questions)
print(total_questions)

291920
2940542
